### 학습 모델 Inference

In [ ]:
import os
import torch
from dataclasses import dataclass
import pandas as pd
from transformers import (
    WhisperForConditionalGeneration, WhisperProcessor
)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model_id = "openai/whisper-tiny"
checkpoint_dir = f"/workspace/results/whisper_train/{model_id.split('/')[-1]}/ -10000"
lang = "ko"
task = "transcribe"

test_df = pd.read_csv("/workspace/kru_data/test.csv")
sample_file = test_df.iloc[0]["abs_path"]
# sample_file = "/workspace/kor_med_stt_data/snu_data/말이좀늦는것같아요_샘플데이터셋/남학생의사_여환자_1.wav"

In [ ]:
import librosa
waveform, sr = librosa.load(sample_file, sr=16000)
# 처음 5초는 넘어가고 5~35초 가지고오기
start_sec = 0
end_sec = 3
start_sample = start_sec * sr
end_sample = end_sec * sr
if len(waveform) > end_sample:
    print(f"This file length is {len(waveform)/sr} seconds. Using audio from {start_sec} to {end_sec} seconds.")
    waveform = waveform[start_sample:end_sample]
else:
    print(f"This file length is {len(waveform)/sr} seconds. Using available audio from {start_sec} seconds to end.")
    waveform = waveform[start_sample:]
    
# sampling된 오디오 jupyter 에서 듣기
import IPython.display as ipd
ipd.Audio(waveform, rate=sr)

In [ ]:
processor = WhisperProcessor.from_pretrained(
    model_id, language="ko", task="transcribe"
)
# model = WhisperForConditionalGeneration.from_pretrained(checkpoint_dir).to("cuda")
model = WhisperForConditionalGeneration.from_pretrained(model_id).to("cuda")

In [ ]:
def transcribe_long_audio(processor, model, waveform, sr, chunk_sec=5):
    chunk_size = chunk_sec * sr
    texts = []
    last_text = ""

    for start in range(0, len(waveform), chunk_size):
        end = start + chunk_size
        chunk = waveform[start:end]
        if len(chunk) == 0:
            continue

        # Whisper inference
        input_features = processor.feature_extractor(
            chunk,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features.to("cuda")

        with torch.no_grad():
            pred_ids = model.generate(input_features)

        text = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)[0]

        # 🔥 중복 제거
        if last_text:
            # 마지막 15자 기준으로 중복 검사
            overlap = last_text[-15:]
            if text.startswith(overlap):
                text = text[len(overlap):]

        texts.append(text)
        last_text = text

    return " ".join(texts).strip()

full_text = transcribe_long_audio(
    processor,
    model,
    waveform,
    sr,
    chunk_sec=3,     # 여기서 원하는 길이 설정
)

print(full_text)

## 모델 평가

In [ ]:
from utils.eval import fast_asr_metrics
import json 
import pandas as pd

# omniasr inference model
# file_path = "/workspace/results/omniasr_inference/omniasr_ctc/omniASR_CTC_300M/test_pred.parquet" # omni asr 

# whisper train model 
file_path = "/workspace/results/whisper_test/whisper-tiny/checkpoint-10000/test_pred.parquet" # whisper train model
# whisper inference model
# file_path = "/workspace/results/whisper_inference/whisper_tiny_inference/test_pred.parquet" # whisper inference model

save_path = file_path.replace(file_path.split("/")[-1], "metrics.json") # file_path 의 디렉토리
df = pd.read_parquet(file_path)

metrics = fast_asr_metrics(df, gt_col = "gt_text", pred_col = "pred_text")
print(metrics)

with open(save_path, "w") as f:
    json.dump(metrics, f, indent=4)
    print(f"Metrics saved to {save_path}")

### 전체 평가

In [2]:
from utils.eval import fast_asr_metrics
import json 
import pandas as pd
from glob import glob 
# test_root_dir = "/workspace/results/"
test_root_dir = "/workspace/kor_med_stt_data/results/whisper_test/whisper-medium"

# test_root_dir 내 모든 test_pred.parquet 파일을 재귀적으로 찾기
test_files = sorted(glob(f"{test_root_dir}/**/test_pred.parquet", recursive=True))
all_list = []
for file_path in test_files:
    save_path = file_path.replace(file_path.split("/")[-1], "metrics.df") # file_path 의 디렉토리
    df = pd.read_parquet(file_path)

    metrics = fast_asr_metrics(df, gt_col = "gt_text", pred_col = "pred_text")
    metrics['model_name'] = file_path.split("/")[-2]
    print(file_path.split("/")[-2])
    print(metrics)

    # metrics를 데이터프레임으로 변환 후 csv로 저장
    
    df_metrics = pd.DataFrame([metrics])
    df_metrics = df_metrics.sort_values(by='wer_mean', ascending=True)
    df_metrics.to_csv(save_path, index=False)
    print(f"Metrics dataframe saved to {save_path}")
    
    all_list.append(metrics)

all_df = pd.DataFrame(all_list)
# 'model_name' 컬럼이 가장 첫 번째 컬럼으로 오도록 순서 조정
cols = ['model_name'] + [col for col in all_df.columns if col != 'model_name']
all_df = all_df[cols]
all_df = all_df.sort_values(by='wer_mean', ascending=True)
all_df.to_csv(f"{test_root_dir}/all_metrics.csv", index=False)
all_df.round(3)

checkpoint-1018124
{'wer_all': 0.7354223130933701, 'cer_all': 0.2882213275523898, 'wer_mean': 0.7236693159795979, 'wer_std': 6.723327342917463, 'wer_median': 0.0, 'wer_min': 0.0, 'wer_max': 1800.0, 'cer_mean': 0.2840517805139876, 'cer_std': 6.440914913864843, 'cer_median': 0.0, 'cer_min': 0.0, 'cer_max': 2950.0, 'inference_time_sec_mean': 0.23499111199137507, 'inference_time_sec_std': 0.05678054619251264, 'inference_time_sec_median': 0.21762687961260477, 'inference_time_sec_min': 0.1138710230588913, 'inference_time_sec_max': 0.4552684525648753, 'num_samples': 254457, 'model_name': 'checkpoint-1018124'}
Metrics dataframe saved to /workspace/kor_med_stt_data/results/whisper_test/whisper-medium/checkpoint-1018124/metrics.df
checkpoint-1272655
{'wer_all': 0.6292702151435527, 'cer_all': 0.2323410566994393, 'wer_mean': 0.6170903188904378, 'wer_std': 6.311219512490857, 'wer_median': 0.0, 'wer_min': 0.0, 'wer_max': 1800.0, 'cer_mean': 0.23075193981146724, 'cer_std': 6.2792593730377915, 'cer_me

,model_name,wer_all,cer_all,wer_mean,wer_std,wer_median,wer_min,wer_max,cer_mean,cer_std,cer_median,cer_min,cer_max,inference_time_sec_mean,inference_time_sec_std,inference_time_sec_median,inference_time_sec_min,inference_time_sec_max,num_samples
1,checkpoint-1272655,0.629,0.232,0.617,6.311,0.0,0.0,1800.0,0.231,6.279,0.0,0.0,2950.0,0.181,0.024,0.189,0.135,0.296,254445
0,checkpoint-1018124,0.735,0.288,0.724,6.723,0.0,0.0,1800.0,0.284,6.441,0.0,0.0,2950.0,0.235,0.057,0.218,0.114,0.455,254457
4,checkpoint-763593,0.966,0.390,0.968,10.912,0.0,0.0,1800.0,0.389,8.665,0.0,0.0,2950.0,0.177,0.011,0.175,0.153,0.273,254445
3,checkpoint-509062,1.298,0.533,1.265,8.506,0.0,0.0,1800.0,0.488,6.962,0.0,0.0,2950.0,0.212,0.036,0.205,0.152,0.778,254445
2,checkpoint-254531,2.129,0.791,2.086,9.854,0.0,0.0,1800.0,0.757,7.764,0.0,0.0,2950.0,0.176,0.040,0.154,0.134,0.488,254445


In [3]:
all_df = all_df.sort_values(by='model_name', ascending=True)

import datetime

def seconds_to_h_m_s(seconds):
    # Converts seconds to HH:MM:SS, never shows days!
    seconds = int(seconds)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds_left = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds_left:02}"

# Extract training time in seconds and convert to HH:MM:SS format (never with days)
training_seconds = [int(name.split("-")[-1]) for name in all_df["model_name"]]
training_hms = [seconds_to_h_m_s(sec) for sec in training_seconds]

# 컬럼 재정의 및 데이터 생성
display_df = pd.DataFrame({
    "Epoch": range(1, len(all_df)+1),
    "CER ↓ (mean ± std)": all_df["cer_mean"].round(3).astype(str) + " ± " + all_df["cer_std"].round(3).astype(str),
    "WER ↓ (mean ± std)": all_df["wer_mean"].round(3).astype(str) + " ± " + all_df["wer_std"].round(3).astype(str),
    "Inference Time ↓(mean ± std, Sec)": all_df["inference_time_sec_mean"].round(3).astype(str) + " ± " + all_df["inference_time_sec_std"].round(3).astype(str),
    "Training Time(Hours:Minutes:Seconds)": training_hms
})
display_df

,Epoch,CER ↓ (mean ± std),WER ↓ (mean ± std),"Inference Time ↓(mean ± std, Sec)",Training Time(Hours:Minutes:Seconds)
0,1,0.284 ± 6.441,0.724 ± 6.723,0.235 ± 0.057,282:48:44
1,2,0.231 ± 6.279,0.617 ± 6.311,0.181 ± 0.024,353:30:55
2,3,0.757 ± 7.764,2.086 ± 9.854,0.176 ± 0.04,70:42:11
3,4,0.488 ± 6.962,1.265 ± 8.506,0.212 ± 0.036,141:24:22
4,5,0.389 ± 8.665,0.968 ± 10.912,0.177 ± 0.011,212:06:33


In [ ]:
import datetime

def hh_mm_ss_to_seconds(time_str):
    # Split off microseconds if present
    if '.' in time_str:
        time_str, micro = time_str.split('.')
    else:
        micro = '0'
    t = datetime.datetime.strptime(time_str, "%H:%M:%S")
    delta = datetime.timedelta(
        hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=int(micro)
    )
    return delta.total_seconds()

def second_to_hh_mm_ss(seconds):
    return datetime.timedelta(seconds=seconds)

second = hh_mm_ss_to_seconds("10:49:38.484706")
print(second)

hh_mm_ss = second_to_hh_mm_ss(second * 1 / 5)
print(f"{hh_mm_ss}")


In [ ]:
pd.read_parquet("/workspace/results/omniasr_inference/omniasr_ctc/omniASR_CTC_1B/test_pred.parquet")

In [26]:
from glob import glob 
import pandas as pd 


result_files = glob("/workspace/results/omniasr_inference/omniasr_ctc/**/metrics.df")

all_list = []
for result_file in result_files:
    model_name = result_file.split("/")[-2]
    df = pd.read_csv(result_file)
    df['model_name'] = model_name
    all_list.append(df)

# model_name이 맨앞 열에 오도록 각 DataFrame의 열 순서 조정
for i, df in enumerate(all_list):
    if 'model_name' in df.columns:
        cols = df.columns.tolist()
        cols.insert(0, cols.pop(cols.index('model_name')))
        all_list[i] = df[cols]

all_df = pd.concat(all_list)
all_df = all_df.sort_values(by='wer_mean', ascending=True)
# 원하는 칼럼: model_name, CER ↓(mean ± std), WER ↓(mean ± std), Inference Time ↓(mean ± std, Sec)

def format_pm(mean, std):
    # float: 소수점 셋째 자리까지 표시
    return f"{mean:.3f} ± {std:.3f}"

# 새로운 컬럼 생성
all_df['CER ↓(mean ± std)'] = all_df.apply(lambda x: format_pm(x['cer_mean'], x['cer_std']), axis=1)
all_df['WER ↓(mean ± std)'] = all_df.apply(lambda x: format_pm(x['wer_mean'], x['wer_std']), axis=1)
if 'inference_time_sec_mean' in all_df.columns and 'inference_time_sec_std' in all_df.columns:
    all_df['Inference Time ↓(mean ± std, Sec)'] = all_df.apply(
        lambda x: format_pm(x['inference_time_sec_mean'], x['inference_time_sec_std']), axis=1
    )
else:
    all_df['Inference Time ↓(mean ± std, Sec)'] = "-"

# 보여줄 컬럼 순서 지정
final_cols = [
    'model_name',
    'CER ↓(mean ± std)',
    'WER ↓(mean ± std)',
    'Inference Time ↓(mean ± std, Sec)'
]

all_df[final_cols].sort_values(by='WER ↓(mean ± std)', ascending=False)

,model_name,CER ↓(mean ± std),WER ↓(mean ± std),"Inference Time ↓(mean ± std, Sec)"
0,omniASR_CTC_300M,26.572 ± 25.709,69.229 ± 28.801,0.073 ± 0.047
0,omniASR_CTC_1B,20.513 ± 20.685,59.631 ± 29.609,0.187 ± 0.068
0,omniASR_CTC_3B,20.461 ± 23.579,57.483 ± 30.922,0.456 ± 0.154
0,omniASR_CTC_7B,19.957 ± 23.730,56.609 ± 30.625,0.745 ± 0.260
